# Worteinbettung und Sprachmodelle
Allen Vektorraum-Modellen ist gemeinsam, dass sie weder Semantik von Wörtern noch Ähnlichkeit erfassen. In diesem Abschnitt werden Worteinbettungen bzw. Embeddings verwendet, um die Bedeutung von Wörtern auf Basis des gemeinsamen Auftretens mit anderen Wörtern zu kodieren.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

## Inhaltsverzeichnis
- [Word2Vec](#Word2Vec)
- [Implementierung](#Implementierung)
- [Zusammenfassung](#Zusammenfassung)

## Word2Vec
Word2Vec ist ein Algorithmus zur Darstellung von Wörtern als Vektoren. Die Grundidee hinter Word2Vec ist, dass Wörter, die in ähnlichen Kontexten verwendet werden, ähnliche Vektoren haben sollten. Die Bedeutung eines Wortes wird damit mit Machine Learning aus den anderen Worten gelernt, mit denen es häufig zusammen auftritt, sodass ein großer Korpus notwendig ist. Die entstehenden Vektoren können dann verwendet werden, um semantische Beziehungen zwischen Wörtern zu erfassen.

Es gibt zwei Hauptarchitekturen, die von Word2Vec verwendet werden, um diese Vektoren zu lernen.

### Continuous Bag of Words (CBOW)
Bei dieser Methode wird das Modell darauf trainiert, ein Wort (das Zielwort) basierend auf dem Kontext (den umgebenden Wörtern) vorherzusagen. Es nimmt eine bestimmte Anzahl von Wörtern vor und nach dem Zielwort und versucht, das Zielwort aus diesen Kontextwörtern zu rekonstruieren.

**Beispiel:**
`Das Auto fährt auf der Straße.`

Wenn das aktuelle Zielwort `fährt` ist, dann wird das Modell darauf trainiert dieses Zielwort auf Basis des Kontexts `Das`, `Auto` und `auf`, `der`, `Straße` vorherzusagen. (Die Fenstergröße lässt sich dabei beeinflussen, sodass kleinere oder größere Teile des Kontexts einfließen.)

### Skip-Gram
Im Gegensatz zum CBOW-Modell geht es beim Skip-Gram-Modell darum, ein Zielwort zu verwenden, um den Kontext vorherzusagen. Es wird also ein Wort (das Zielwort) genommen und versucht, die umgebenden Kontextwörter zu bestimmen.

**Beispiel**: `Das Auto fährt auf der Straße.`

Wenn das aktuelle Zielwort erneut `fährt` ist, dann wird das Modell darauf trainiert, die in der Umgebung befindlichen Kontextwörter `Das`, `Auto`, `auf`, `der` und `Straße` zu rekonstruieren. (Erneut lässt sich die Fenstergröße anpassen.)

### Umsetzung
Zur Umsetzung wird ein neuronales Netz trainiert, das in der Regel mit einer versteckten Schicht auskommt. Die Ein- und Ausgabeschichten hängen dann von der verwendeten Architektur ab.

| Architektur | Eingabe                                   | Ausgabe                                               |
| ----------- | ----------------------------------------- | ----------------------------------------------------- |
| CBOW        | 1 One-Hot-kodierter Vektor des Zielwortes | n Wahrscheinlichkeitsvektoren (*SoftMax*) für Kontext |
| Skip-Gram   | n One-Hot-kodierte Vektoren des Kontexts  | 1 Wahrscheinlichkeitsvektor (*SoftMax*) für Zielwort  |

Interessant ist aber in beiden Fällen die versteckte Schicht. Diese ist eine lineare Transformation des oder der Eingabevektoren und führt am Ende zur gewünschten Vorhersage. Eingabewörter, die eine ähnliche semantische Bedeutung haben und damit in einem ähnlichen Kontext verwendet werden, müssen also nach dem Training zwangsweise eine ähnliche Zwischenrepräsentation erzeugen.

Wird diese Zwischenrepräsentation - die Ausgabe der versteckten Schicht nach einer Aktivierung - als Vektor interpretiert, werden in ähnlichem Kontext verwendete Wörter auf ähnliche Vektoren abgebildet, die als Worteinbettungen (*Word Embeddings*) bezeichnet werden. (Die Größe der versteckten Schicht bestimmt somit direkt die Größe des Vektors und damit die Dimension des Vektorraums.)

## Implementierung
Definition und Training neuronaler Netze werden an anderer Stelle gelehrt. Stattdessen soll eine fertige Implementierung aus dem Paket [Gensim](https://pypi.org/project/gensim/) an folgendem Beispieltext veranschaulicht werden.

In [ ]:
sentences = [
    ["der", "rechner", "spinnt", "schon", "wieder"],
    ["ich", "habe", "mir", "einen", "neuen", "computer", "gekauft"],
    ["der", "pc", "vom", "arbeitskollegen", "ist", "besser", "als", "meiner"],
    ["der", "laptop", "funktioniert", "nicht", "richtig"],
    ["ich", "habe", "mir", "ein", "neues", "smartphone", "besorgt"],
    ["der", "drucker", "ist", "kaputt", "gegangen"],
    ["mein", "tablet", "ist", "sehr", "langsam", "geworden"],
    ["der", "monitor", "von", "meinem", "freund", "ist", "größer", "als", "meiner"],
    ["ich", "muss", "mein", "operating", "system", "aktualisieren"],
    ["mein", "speicher", "ist", "fast", "voll"],
    ["die", "kamera", "auf", "dem", "neuen", "handy", "ist", "sehr", "gut"],
    ["der", "pc", "von", "meinem", "bruder", "läuft", "schneller", "als", "mein", "computer"],
    ["ich", "habe", "mein", "wifi", "gerade", "auf", "ein", "besseres", "netzwerk", "umgestellt"],
    ["mein", "laptop", "hat", "schon", "sehr", "viele", "jahre", "auf", "dem", "buckel"],
    ["die", "neue", "software", "auf", "meinem", "computer", "läuft", "flüssiger"],
    ["der", "akku", "meines", "laptops", "hält", "nicht", "mehr", "lange"],
    ["der", "speicher", "meines", "smartphones", "reicht", "nicht", "mehr", "für", "alle", "meine", "bilder"],
    ["mein", "freund", "hat", "sich", "eine", "neue", "grafikkarte", "gekauft"],
    ["die", "website", "lässt", "sich", "auf", "meinem", "browser", "nicht", "laden"],
    ["ich", "habe", "meinen", "pc", "aufgerüstet", "mit", "mehr", "ram"],
    ["mein", "laptop", "hat", "einen", "defekten", "bildschirm"],
]

Das Training wird durch die Klasse `Word2Vec` abstrahiert.

In [ ]:
model = Word2Vec(sentences=sentences, vector_size=4, window=5, min_count=1)

Über das Attribut `wv` (**Word Vectors**) lassen sich die Vektoren abfragen.

In [ ]:
model.wv['computer']

In [ ]:
model.wv['smartphone']

In [ ]:
model.wv['arbeitskollegen']

Außerdem ist die Suche nach ähnlichen Vektoren und damit ähnlichen Wörtern bereits integriert.

In [ ]:
model.wv.most_similar('computer')

## Zusammenfassung
Worteinbettungen beziehen im Gegensatz zu den zuvor gezeigten Methoden auch den Kontext mit ein und schaffen es so, ähnlich verwendete Wörter zu ähnlichen Vektoren zu transformieren. Auch hier sollten Sie den Code erneut mit einem größeren Korpus und ggf. einer ausgefeilteren Vorverarbeitung probieren. (Übliche Vektorgrößen liegen im Bereich von $100$ bis $300$. Manchmal verbessern auch Größen bis zu $1.000$ noch die Genauigkeit spürbar.)